In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import numpy as np
import cv2
import os
import json
import random
import torch.nn as nn
from torchvision.models.segmentation import deeplabv3_resnet50

# Configuración de dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformaciones de imagen
data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Función para eliminar el fondo (como en el entrenamiento)
def remove_background(image):
    image_np = np.array(image)
    gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
    _, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.zeros_like(gray)

    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        cv2.drawContours(mask, [largest_contour], -1, (255), thickness=cv2.FILLED)

    result = cv2.bitwise_and(image_np, image_np, mask=mask)
    return Image.fromarray(result)

# Modelo HRNet-W32 para predicción de poses
class HRNetForPose(nn.Module):
    def __init__(self, num_keypoints=102):
        super(HRNetForPose, self).__init__()
        self.backbone = deeplabv3_resnet50(pretrained=True).backbone
        self.fc = nn.Sequential(
            nn.Conv2d(2048, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, num_keypoints, kernel_size=1, stride=1),
            nn.AdaptiveAvgPool2d((1, 1))
        )

    def forward(self, x):
        x = self.backbone(x)['out']
        x = self.fc(x)
        return x.view(x.size(0), -1)

# Cargar pesos del modelo
model = HRNetForPose().to(device)
model.load_state_dict(torch.load("hrnet_best_model.pth", map_location=device))
model.eval()

# Inicializar filtro de Kalman con mayor suavizado
def initialize_kalman():
    kalman = cv2.KalmanFilter(6, 2)
    kalman.measurementMatrix = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]], np.float32)
    kalman.transitionMatrix = np.array([
        [1, 0, 1, 0, 0.5, 0], [0, 1, 0, 1, 0, 0.5],
        [0, 0, 1, 0, 1, 0], [0, 0, 0, 1, 0, 1],
        [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]
    ], np.float32)
    kalman.processNoiseCov = np.eye(6, dtype=np.float32) * 0.0005  # Más suavidad
    return kalman

# Aplicar filtro de Kalman a coordenadas
def apply_kalman_filter(kalman_filters, points):
    filtered_points = []
    num_joints = len(points) // 3

    if len(kalman_filters) < num_joints:
        kalman_filters.extend([initialize_kalman() for _ in range(num_joints - len(kalman_filters))])

    for i in range(0, len(points), 3):
        x, y, c = float(points[i]), float(points[i + 1]), float(points[i + 2])  # Asegurar que sean floats
        if c > 0.5 and (i // 3) < len(kalman_filters):
            measurement = np.array([[np.float32(x)], [np.float32(y)]])
            kalman_filters[i // 3].correct(measurement)
            predicted = kalman_filters[i // 3].predict()
            filtered_points.extend([predicted[0][0], predicted[1][0], c])
        else:
            filtered_points.extend([x, y, c])
    return filtered_points

# Colores para los dos peleadores (Rojo y Azul)
fighter_colors = [(0, 0, 255), (255, 0, 0)]  # Azul para peleador 1, Rojo para peleador 2

# Dibujar solo puntos, sin uniones
def draw_pose(image, pose, kalman_filters, colors):
    pose = apply_kalman_filter(kalman_filters, pose)
    num_joints = len(pose) // 3

    for i in range(num_joints):
        x, y, c = int(pose[i * 3] * 256), int(pose[i * 3 + 1] * 256), pose[i * 3 + 2]
        if c > 0.5:
            cv2.circle(image, (x, y), 2, colors[i // (num_joints // 2)], -1)  # Rojo para uno, Azul para el otro
    return image

# Directorios de imágenes
annotations_path_v2 = "./mnt/V2/annotations/annotations_preprocessed.json"
image_dir_v2 = "./mnt/V2/images/"
image_dir_v3 = "./mnt/V3/images/"
output_video_path = "pose_comparison.mp4"
frame_rate = 15
num_frames = 500

def create_video():
    with open(annotations_path_v2, 'r') as f:
        annotations = json.load(f)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (512, 256))

    start_idx = random.randint(0, max(0, len(annotations) - num_frames))

    kalman_filters_gt = [initialize_kalman() for _ in range(34)]
    kalman_filters_pred = [initialize_kalman() for _ in range(34)]

    for annotation in annotations[start_idx:start_idx + num_frames]:
        image_name = annotation['Image'] + '.jpg'
        image_path_v2 = os.path.join(image_dir_v2, image_name)
        image_path_v3 = os.path.join(image_dir_v3, image_name)

        if not os.path.exists(image_path_v2) or not os.path.exists(image_path_v3):
            print(f"Imagen no encontrada: {image_name}")
            continue

        image = cv2.imread(image_path_v2)
        image = cv2.resize(image, (256, 256))

        pose_gt = sum(annotation['Pose1'], []) + sum(annotation['Pose2'], [])  # Aplanar lista
        image_gt = draw_pose(image.copy(), pose_gt, kalman_filters_gt, fighter_colors)

        image_pil = remove_background(Image.open(image_path_v3).resize((256, 256)))
        image_tensor = data_transforms(image_pil).unsqueeze(0).to(device)
        pose_pred = model(image_tensor).cpu().detach().numpy().flatten()

        image_pred = draw_pose(image.copy(), pose_pred, kalman_filters_pred, fighter_colors)

        video_writer.write(np.hstack((image_gt, image_pred)))

    video_writer.release()
    print(f'Video guardado en {output_video_path}')

# Ejecutar
create_video()


FileNotFoundError: [Errno 2] No such file or directory: './mnt/V2/annotations/annotations_preprocessed.json'